In [1]:
import numpy as np
import matplotlib.pylab as plot
from astropy.io import ascii,fits
from scipy import interpolate
import grb_catalogs_copy
from BurstCube.LocSim.Detector import *
from BurstCube.LocSim.Spacecraft import *
from astropy.coordinates import SkyCoord
from astropy import units as u
from scipy.optimize import curve_fit

In [13]:
# read in GBM GRB catalog (first 9 years)
gbm=fits.open('gbmgrbcat_copy.fits')
gbm=gbm[1].data

In [14]:
## read in GBM effective area curve
def getGBMdata(gbmfile=None):
    """Reads the GBM NaI effective area file and returns a numpy array
    with two columns ``energy`` and ``aeff``.
    Parameters
    ----------
    gbmfile : string
       Name of file that contains the GBM data.
    Returns
    ----------
    gbmdata : array 
    numpy array with two columns ``energy`` and ``aeff``
    """

   # from numpy import genfromtxt
    #from os import path

    #gbmfile = path.expandvars(gbmfile)
    
    return np.genfromtxt(gbmfile,skip_header=2,names=('energy', 'aeff'))


In [15]:
## bit of useful code for interpolating in log space
def loginterpol(x,y,x1):

    f=interpolate.interp1d(np.log10(x),np.log10(y),bounds_error=False,fill_value="extrapolate",kind='linear')
    y1=10**f(np.log10(x1))

    return y1

def loginterpol2d(x,y,z,x1,y1):

    wz=np.where(z==0)[0]
    zz=z
    zz[wz]=1.
    f=interpolate.interp2d(x,y,np.log10(zz),bounds_error=False,fill_value="extrapolate",kind='linear')
    z1=10**f(x1,y1)

In [16]:
## generate random positions on the sky with equal area probability
def random_sky(n=1):

    u=np.random.rand(n)
    v=np.random.rand(n)

    phi=2*np.pi*u
    theta=np.arccos(2*v-1.)

    dec=-np.degrees(theta-np.pi/2.)
    ra=np.degrees(np.pi*2-phi)

    return ra,dec

In [17]:
## read in the GBM Aeff
aeff_gbm = getGBMdata('/home/alyson/NASA/Simulation/BurstCube/Users/ajoens/gbm_effective_area.dat')
print(aeff_gbm)

[(8.10939400e+00,   6.043869 ) (8.27917900e+00,   7.1733994)
 (8.54091550e+00,   9.051345 ) (8.71935400e+00,  11.009052 )
 (9.09046600e+00,  13.555708 ) (9.47675300e+00,  17.315489 )
 (9.77528500e+00,  23.22683  ) (1.05165080e+01,  30.780512 )
 (1.16751795e+01,  43.367252 ) (1.26926560e+01,  58.89598  )
 (1.50095340e+01,  80.0014   ) (1.83216400e+01,  97.346825 )
 (2.26054130e+01, 106.105095 ) (3.13079800e+01, 120.00855  )
 (3.26618730e+01, 102.37214  ) (4.11476300e+01, 123.06248  )
 (8.95676700e+01, 132.68503  ) (1.45272660e+02, 131.2247   )
 (3.44349330e+02,  75.82555  ) (1.07285790e+03,  42.783394 )
 (2.76511250e+03,  30.818037 ) (9.45941300e+03,  32.459995 )]


In [10]:
## separate GBM short & long GRBs
w=np.where(gbm['FLUX_1024']>0)
gbm=gbm[w]
s=np.where(gbm['T90'] <= 2.)[0]
l=np.where(gbm['T90'] > 2.)[0]
m=np.where(gbm['PFLX_BEST_FITTING_MODEL'][s] == ' ')

In [19]:
## grab short GRBs with peak spectral info
# s is defined twice. do we define s as where flux_1024 is greater than 0 or as the below?
# or is this for burstcube and above for GBM?
#s=np.where((gbm['T90'] <= 2.) & (gbm['PFLX_BEST_FITTING_MODEL'] != ' '))[0]
#eng=aeffs['keV']
#engdiff=eng[1:]-eng[0:-1]
#w=np.where(engdiff<0)[0]
#nsims=len(w)
#w=np.append(-1,w)#,len(eng)

In [21]:
#Integrating the best fit spectrum for each GRB in the energy range of 50-300 KeV. 
#This will give us the photon flux in units of ph/cm^2/s. Currently only doing this for GBM and will then add in BurstCube
# the parts below I have commented out are what I believe are for BC and currently I am focusing on GBM
mo=gbm['PFLX_BEST_FITTING_MODEL'][s]
#f=np.zeros([len(s),nsims]) # produces an array of zeros with the given shape and type
pf=np.zeros(len(s))
outE=np.logspace(np.log10(50),np.log10(300),20) # returns numbers spaced evenly on a log scale
for i in range(len(s)):
    #for j in range(nsims):
        #E=np.array(eng[w[j]+1:w[j+1]+1])
        #AeffBC=loginterpol(E,aeffs['aeff'][w[j]+1:w[j+1]+1],outE)
        AeffGBM=loginterpol(aeff_gbm['energy'],aeff_gbm['aeff'],outE)#eng[w[j]+1:w[j+1]+1])
        #Aratio=(AeffBC/AeffGBM)
        # not sure what *grb_catalogs_copy.pl(outE,gbm['PFLX_PLAW_INDEX'][s[i]] is and why we need it. I think we only need the model photon flux times the aeffGBM and we want it integrated over the energy range provided in outE
        # this should give us an array of the maximum observed photon flux for GBM
        if mo[i]=='PFLX_PLAW':
            pf[i]=np.trapz(gbm['PFLX_PLAW_AMPL'][s[i]]*grb_catalogs_copy.pl(outE,gbm['PFLX_PLAW_INDEX'][s[i]])*AeffGBM,outE)
            #pf[i]=gbm['PFLX_PLAW_PHTFLUX'][s[i]]
        if mo[i]=='PFLX_COMP':
            pf[i]=np.trapz(gbm['PFLX_COMP_AMPL'][s[i]]*grb_catalogs_copy.comp(outE,gbm['PFLX_COMP_INDEX'][s[i]],gbm['PFLX_COMP_EPEAK'][s[i]])*AeffGBM,outE)
            #pf[i]=gbm['PFLX_COMP_PHTFLUX'][s[i]]
        if mo[i]=='PFLX_BAND':
            pf[i]=np.trapz(gbm['PFLX_BAND_AMPL'][s[i]]*grb_catalogs_copy.band(outE,gbm['PFLX_BAND_ALPHA'][s[i]],gbm['PFLX_BAND_EPEAK'][s[i]],gbm['PFLX_BAND_BETA'][s[i]])*AeffGBM,outE)
            #pf[i]=gbm['PFLX_BAND_PHTFLUX'][s[i]]
        if mo[i]=='PFLX_SBPL':
            pf[i]=np.trapz(gbm['PFLX_SBPL_AMPL'][s[i]]*grb_catalogs_copy.sbpl(outE,gbm['PFLX_SBPL_INDX1'][s[i]],gbm['PFLX_SBPL_BRKEN'][s[i]],gbm['PFLX_SBPL_INDX2'][s[i]])*AeffGBM,outE)
            #pf[i]=gbm['PFLX_SBPL_PHTFLUX'][s[i]]
            
print(pf)

[  280.94468472   243.66864428   282.76066087   590.63703052
   183.33690011   468.8158535    652.09032474  1382.29679242
   151.53885503   618.99521694   111.23824754  1041.51962443
   161.36368401   260.724765     364.74499756   364.14407
   127.40779999   174.27828982   650.48161621   322.92445403
   277.97218119   418.41327323   174.88396444   307.176089
   165.02763044   429.0033272    217.24666128  1019.81641782
  2510.76514696   533.24609952   209.76464697  2086.4978415
   187.98772731   282.82759168   149.7802356   1003.39472644
   462.05114741   454.56714774   207.78293428  7192.02224001
   770.52483529   331.67471986   186.88923448   490.74257817
  1249.93572569   262.85275705   296.82245612   306.72991036
   123.39102923   209.84702463   617.72864428   524.45706308
   200.14025286   455.57732735  1079.44254907   263.86769274
   428.14087837   696.0543342    297.8787065    355.52281081
   197.65621378   351.84212234   314.05809971   752.43194341
   237.94873121   651.88619143